In [19]:
#from numpy import array

from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error, r2_score
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.core import Dropout
from keras.layers import LSTM

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg



#load dataset
dataset = read_csv('phy_cps.csv', header=0, index_col=0)
values = dataset.values
#integer encode wind direction, as it's the only categorical variable.
#encoder = LabelEncoder()
#values[:,4] = encoder.fit_transform(values[:,4])
 
#ensure all data are float32 values
values = values.astype('float32')
#normalize input features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

#frame as supervised learning
n_hours = 3
n_features = 6 
reframed = series_to_supervised(scaled, n_hours, 1)
values = reframed.values
n_train_hours = 1936
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]

#split into input and outputs
n_obs = n_hours * n_features
train_X = train[:, :n_obs]
train_y = train[:, -n_features:(-n_features+4)] #+2 because of indexing madness.
test_X = test[:, :n_obs]
test_y = test[:, -n_features:(-n_features+4)]


train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))

#Need to output four values, not one.
#design network
model = Sequential()
model.add(LSTM(50, activation='relu',return_sequences=True, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dropout(0.1))
model.add(LSTM(20, activation='relu', return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(20, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(4))

model.compile(loss='mse', optimizer='adam')
 
#fit network
history = model.fit(train_X, train_y, epochs=200, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)

# Plot history
pyplot.xlabel('Epochs')
pyplot.ylabel('Loss')
pyplot.title('Model Train vs Testing Loss')
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

#make a prediction
y_hat = model.predict(test_X)

Epoch 1/200
27/27 - 7s - loss: 0.2286 - val_loss: 0.1764 - 7s/epoch - 266ms/step
Epoch 2/200
27/27 - 0s - loss: 0.1964 - val_loss: 0.1465 - 282ms/epoch - 10ms/step
Epoch 3/200
27/27 - 0s - loss: 0.1325 - val_loss: 0.1104 - 266ms/epoch - 10ms/step
Epoch 4/200
27/27 - 0s - loss: 0.1069 - val_loss: 0.1046 - 258ms/epoch - 10ms/step
Epoch 5/200
27/27 - 0s - loss: 0.0892 - val_loss: 0.0919 - 259ms/epoch - 10ms/step
Epoch 6/200
27/27 - 0s - loss: 0.0722 - val_loss: 0.0770 - 258ms/epoch - 10ms/step
Epoch 7/200
27/27 - 0s - loss: 0.0623 - val_loss: 0.0656 - 260ms/epoch - 10ms/step
Epoch 8/200
27/27 - 0s - loss: 0.0572 - val_loss: 0.0553 - 279ms/epoch - 10ms/step
Epoch 9/200
27/27 - 0s - loss: 0.0573 - val_loss: 0.0550 - 299ms/epoch - 11ms/step
Epoch 10/200
27/27 - 0s - loss: 0.0519 - val_loss: 0.0531 - 258ms/epoch - 10ms/step
Epoch 11/200
27/27 - 0s - loss: 0.0484 - val_loss: 0.0496 - 258ms/epoch - 10ms/step
Epoch 12/200
27/27 - 0s - loss: 0.0432 - val_loss: 0.0486 - 261ms/epoch - 10ms/step
Epo

Epoch 99/200
27/27 - 0s - loss: 0.0108 - val_loss: 0.0053 - 258ms/epoch - 10ms/step
Epoch 100/200
27/27 - 0s - loss: 0.0110 - val_loss: 0.0056 - 252ms/epoch - 9ms/step
Epoch 101/200
27/27 - 0s - loss: 0.0106 - val_loss: 0.0048 - 254ms/epoch - 9ms/step
Epoch 102/200
27/27 - 0s - loss: 0.0099 - val_loss: 0.0054 - 254ms/epoch - 9ms/step
Epoch 103/200
27/27 - 0s - loss: 0.0100 - val_loss: 0.0057 - 253ms/epoch - 9ms/step
Epoch 104/200
27/27 - 0s - loss: 0.0098 - val_loss: 0.0053 - 261ms/epoch - 10ms/step
Epoch 105/200
27/27 - 0s - loss: 0.0094 - val_loss: 0.0051 - 254ms/epoch - 9ms/step
Epoch 106/200
27/27 - 0s - loss: 0.0097 - val_loss: 0.0048 - 266ms/epoch - 10ms/step
Epoch 107/200
27/27 - 0s - loss: 0.0095 - val_loss: 0.0051 - 251ms/epoch - 9ms/step
Epoch 108/200
27/27 - 0s - loss: 0.0095 - val_loss: 0.0045 - 257ms/epoch - 10ms/step
Epoch 109/200
27/27 - 0s - loss: 0.0093 - val_loss: 0.0045 - 253ms/epoch - 9ms/step
Epoch 110/200
27/27 - 0s - loss: 0.0096 - val_loss: 0.0048 - 254ms/epoch 

27/27 - 0s - loss: 0.0074 - val_loss: 0.0038 - 265ms/epoch - 10ms/step
Epoch 197/200
27/27 - 0s - loss: 0.0077 - val_loss: 0.0036 - 255ms/epoch - 9ms/step
Epoch 198/200
27/27 - 0s - loss: 0.0077 - val_loss: 0.0038 - 254ms/epoch - 9ms/step
Epoch 199/200
27/27 - 0s - loss: 0.0075 - val_loss: 0.0039 - 257ms/epoch - 10ms/step
Epoch 200/200
27/27 - 0s - loss: 0.0078 - val_loss: 0.0037 - 256ms/epoch - 9ms/step


In [22]:
#make a prediction
y_hat = model.predict(test_X)

In [23]:
y_hat.shape

(481, 4)

In [ ]:
 



 


 
#calculate RMSE - CHANGED to output RMSE for each variable.
norm_rmse_1 = sqrt(mean_squared_error(y_hat[:,0], test_y[:,0])) #RMSE for the first variable (Tank_1)
norm_rmse_2 = sqrt(mean_squared_error(y_hat[:,1], test_y[:,1])) #RMSE for the second variable (Tank_2)
norm_rmse_3 = sqrt(mean_squared_error(y_hat[:,2], test_y[:,2])) #RMSE for the third variable (Tank_3)
norm_rmse_4 = sqrt(mean_squared_error(y_hat[:,3], test_y[:,3])) #RMSE for the fourth variable (Pump_1)
print('Normalized Test RMSE: ', norm_rmse_1, norm_rmse_2, norm_rmse_3, norm_rmse_4)
    
test_X = test_X.reshape((test_X.shape[0], n_hours*n_features))
inv_yhat = concatenate((y_hat, test_X[:,-2:]), axis=1) #changed 7 to 6
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0:4] #changed from 0 to 0:2. Should be first 2 columns that contain the predictions
 

#invert scaling for actual
test_y = test_y.reshape((len(test_y),4)) #changed 1 to 2
inv_y = concatenate((test_y, test_X[:,-2:]), axis=1) #changed 7 to 6
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0:4] #changed from 0 to 0:2. Should be first 2 columns that contain the predictions.
 

    
#calculate RMSE - CHANGED to output RMSE for each variable.
rmse_1 = sqrt(mean_squared_error(inv_y[:,0], inv_yhat[:,0])) #RMSE for the first variable (Tank_1)
rmse_2 = sqrt(mean_squared_error(inv_y[:,1], inv_yhat[:,1])) #RMSE for the second variable (Tank_2)
rmse_3 = sqrt(mean_squared_error(inv_y[:,2], inv_yhat[:,2])) #RMSE for the third variable (Tank_3)
rmse_4 = sqrt(mean_squared_error(inv_y[:,3], inv_yhat[:,3])) #RMSE for the fourth variable (Pump_1)

r_sq1 = r2_score(inv_y[:,0], inv_yhat[:,0])
r_sq2 = r2_score(inv_y[:,1], inv_yhat[:,1])
r_sq3 = r2_score(inv_y[:,2], inv_yhat[:,2])
r_sq4 = r2_score(inv_y[:,3], inv_yhat[:,3])

print('Test RMSE: ', rmse_1, rmse_2, rmse_3, rmse_4)

print("Test R-squared: ", r_sq1, r_sq2, r_sq3, r_sq4)
model.summary()

# Save the model to disk
model.save('multi_lstm_model.h5')
print('Model saved to disk')
